In [2]:
#dataset create train, test
import math
import os
import sys
import time
import glob
import xlrd
import numpy as np
import pandas as pd
import random


def train_and_test_set():
    """
    ECUST-FBP dataset creation
    """
    
    DATA_DIR = '/home/ubuntu/crop/'
    LABELS_FILE = '/home/ubuntu/test5.xlsx'
    
    
    df = pd.read_excel(LABELS_FILE)  
    labels_dict= {}
    for root, dirs, files in os.walk(DATA_DIR):
     
      for file in files: 
        
        f = os.path.join(root, file)
        df1 = df.loc[df['image']==file]
        if (len(df1.index)==1):
         
           labels_dict[f] = float(df1['mean'])
  
    keys =  list(labels_dict.keys())
    print(len(labels_dict))
    random.shuffle(keys)
    d1 = dict(list(labels_dict.items())[len(labels_dict)//2:])
    d2 = dict(list(labels_dict.items())[:len(labels_dict)//2])
    d3 = dict(list(labels_dict.items())[:len(labels_dict)//2])
    return d1, d2, d3      

train,test,val = train_and_test_set()
with open('train.txt', 'w') as file:
     for k,v in train.items():
            file.write(str(k)+' '+str(v)+'\n')

with open('test.txt', 'w') as file2:
     for k,v in test.items():
            file2.write(str(k)+' '+str(v)+'\n')
            
with open('val.txt', 'w') as file3:
     for k,v in val.items():
            file3.write(str(k)+' '+str(v)+'\n')            
            

XLRDError: Excel xlsx file; not supported

In [4]:
!pip install openpyxl

In [3]:
# get SCUT5500 train test 

import math
import os
import sys
import time

import glob
from xlrd import open_workbook
import numpy as np
import pandas as pd
import random
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
import dask.dataframe as dd
import dask.array as da
from util.calc_util import split_train_and_test_data
from util.cfg import config
from util.file_util import mkdirs_if_not_exist, out_result, prepare_scutfbp5500
from util.vgg_face_feature import extract_feature

from keras.engine import  Model
from keras.layers import Input
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
vgg_model = VGGFace() # pooling: None, avg or max

def scut5500_train_and_test_set():
    """
    SCUT-FBP5500 dataset creation
    """ 

    df = pd.read_csv("train2.txt", sep=" ", header=None)
    df.columns = ['image', 'score']
    train1 = {}
    
    for index, row in df.iterrows():
        img= row['image']
        train1[img] =  float(row['score'] ) 
   
    df = pd.read_csv("/home/ubuntu/test2.txt", sep=" ", header=None)
    df.columns = ['image', 'score']
    test1 = {}
    
    for index, row in df.iterrows():
        img= row['image']
        test1[img] =  float(row['score'] ) 
    return train1, test1
    
train, test = scut5500_train_and_test_set()
train, test

ModuleNotFoundError: No module named 'keras_vggface'

In [2]:
    #  Train, validation and VGGface feature extraction
    
    def extract_feature_tf2(file, layer_name):
   
        out = vgg_model.get_layer(layer_name).output
        vgg_model_new = Model(vgg_model.input, out)
        img = image.load_img(file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = utils.preprocess_input(x, version=1) # or version=2
        features = vgg_model_new.predict(x)
    
        return features.ravel()
    
    
    train_vec = list()
    train_label = list()
    test_vec = list()
    test_label = list()

    for k, v in train.items():

        try:
            feature = np.concatenate((extract_feature_tf2(k, layer_name="fc6"), extract_feature_tf2(k, layer_name="fc7")),
                                 axis=0)
      
            train_vec.append(feature)
            train_label.append(v)
        except:
            print("no file"+k)
    for k, v in test.items():
  
        try:
            feature = np.concatenate((extract_feature_tf2(k, layer_name="fc6"), extract_feature_tf2(k, layer_name="fc7")),
                                 axis=0)
            test_vec.append(feature)
            test_label.append(v)
        except:
            print("no file"+k)
  
    #train the model and validate
    
    
    #train_vec = pca.fit_transform(train_vec)
    #test_vec = pca.fit_transform(test_vec)    
    reg = linear_model.BayesianRidge()
    #print(train_label)
    train_vec = np.array(train_vec)
    train_label = np.array(train_label)
    reg.fit(train_vec, train_label)

    predicted_label = reg.predict(np.array(test_vec))
    mae_lr = round(mean_absolute_error(np.array(test_label), predicted_label), 4)
    rmse_lr = round(math.sqrt(mean_squared_error(np.array(test_label), predicted_label)), 4)
    pc = round(np.corrcoef(test_label, predicted_label)[0, 1], 4)

    print('===============The Mean Absolute Error of Model is {0}===================='.format(mae_lr))
    print('===============The Root Mean Square Error of Model is {0}===================='.format(rmse_lr))
    print('===============The Pearson Correlation of Model is {0}===================='.format(pc))

    csv_tag = time.time()

    mkdirs_if_not_exist('./result')
    df = pd.DataFrame([mae_lr, rmse_lr, pc])
    df.to_csv('./result/performance_%s.csv' % csv_tag, index=False)

    out_result(list(test.keys()), predicted_label.flatten().tolist(), test_label, None,
               path='./result/detail_%s.csv' % csv_tag)

no file/home/ubuntu/mtcnn_aligned_images_SCUT/mtw83.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/mty1704.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty1611.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/mtw9.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty890.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty956.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty1641.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/ftw587.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/ftw669.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/mty1729.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/ftw673.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/ftw726.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty1274.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/ftw592.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty1967.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/mty1699.jpg
no file/home/ubuntu/mtcnn_aligned_images_SCUT/fty1197.jpg
no file/home/ubuntu/mtcnn_

ValueError: Shape of passed values is (3, 1), indices imply (3, 3)

In [3]:
#save the model
import pickle
filename = 'scut55_trans_all_crop.sav'
pickle.dump(reg, open(filename, 'wb'))

In [4]:
i=0
while i < len(predicted_label):
    print(predicted_label[i],test_label[i])
    i=i+1

2.0887281726881683 2.8
2.5783428063928544 2.3666669999999996
2.355161193655504 2.6166669999999996
2.5371307592391625 2.816667
2.694794877098517 2.4
2.8657352162347998 2.5333330000000003
3.220483191301966 2.333333
2.7268656599121597 2.8
3.7866580520582813 4.5
3.3301923831508726 2.75
2.960550960408881 2.9
2.7670634227014705 2.433333
2.912454222489985 2.666667
3.0129349446764504 2.733333
3.2741671941458472 2.9666669999999997
3.7162459708494935 3.583333
2.357379569685593 2.266667
2.1382603162534184 2.2
3.7063184280771315 3.833333
2.9521583357186305 2.8833330000000004
2.236983638355687 2.2
2.5577946276254457 2.4666669999999997
2.967433328734728 2.733333
2.9815107886349876 2.75
1.9700073358517933 1.9333330000000002
3.4920970937719518 3.9
3.630418686980744 4.05
2.857047726343687 2.5333330000000003
2.951463680756069 2.983333
2.8700501262098035 2.6
3.383948624347564 4.0
3.1059365212735317 2.666667
2.624748885698361 2.766667
2.6486704349582677 2.6333330000000004
2.725471295033729 2.983333
3.2283

2.902893203906662 2.9666669999999997
3.888426188909663 4.05
2.1850093378827093 1.9666669999999997
3.4343783735766262 3.6333330000000004
2.2664752869196594 2.433333
2.935753092299775 2.7833330000000003
2.724936888257458 2.95
2.79035768117599 2.8
1.883599743956195 1.9333330000000002
3.401012133403331 3.233333
2.6880277921898146 2.95
3.955246520756413 3.35
3.6496271802070144 4.25
3.6051247962912547 3.8833330000000004
2.985681243268488 2.766667
2.879114187688266 2.583333
2.285413111621825 2.1
3.02153492970903 2.75
2.7244281834013395 2.8
3.195190816499018 3.3
2.4909397816470524 2.65
3.6600637265004377 4.133333
3.5880360339241806 3.4666669999999997
2.5555124934862405 2.483333
2.8034442539842024 2.6166669999999996
3.4246774833221103 4.033333
3.9561338795536325 4.216667
2.727887985402163 2.916667
2.4730731588672756 2.266667
3.0641284425014796 2.65
3.8946862896100085 3.933333
3.1960619434083144 2.666667
2.1231105660136658 2.583333
4.129623770298613 4.033333
1.4188632976040196 1.083333
2.9657525

In [1]:
import os
list = os.listdir('/home/ubuntu/mtcnn_aligned_images_SCUT/') # dir is your directory path
number_files = len(list)
print (number_files)

5405
